In [1]:
%pip install -q ipywidgets
%pip install -q bqplot

In [2]:
import ipywidgets as ipw
import bqplot as bqp
import pandas as pd
import numpy as np
import sympy as sp
from bqplot import pyplot as plt
from scipy import interpolate as numint
from IPython.display import display, HTML
from sympy.printing import latex
from sympy import *

In [3]:
%%html
<div style="text-align: center; font-size: 100pt; width: 100%; color: black">
    <math xmlns="http://www.w3.org/1998/Math/MathML">
        <mrow>
            <mo>&#8747;</mo>
            <mspace width="0.3em"/>
            <mi>vs</mi>
            <mspace width="0.3em"/>
            <mo>&#8721;</mo>
        </mrow>
    </math>
</div>

In [4]:
margin1 = '0px 0px 10px 0px'
width1  = '225px'
HTML1 = ipw.HTML(
        value  = f'<center><font face="Times New Roman"><font size="5">\
                 Parametros de calculo\
                 </font></font></center>',
        layout = ipw.Layout(width = '250px', margin = margin1)
) 
Dropdown1 = ipw.Dropdown(
            options = ['Elastico', 'Fijo'],
            value = 'Fijo',
            description = 'Apoyo 1',
            layout = ipw.Layout(width = width1, margin = margin1)
)
Dropdown2 = ipw.Dropdown(
            options = ['Elastico', 'Fijo'],
            value = 'Fijo',
            description = 'Apoyo 2',
            layout = ipw.Layout(width = width1, margin = margin1)
)
Dropdown3 = ipw.Dropdown(
            options = ['Cortantes', 'Momentos', 'Pendientes', 'Deflexiones'],
            value = 'Cortantes',
            description = 'Grafica de: ',
            layout = ipw.Layout(width = '30%', height = 'auto', margin = '0px 30px 0px 0px')
)
BoundedFloatText1 = ipw.BoundedFloatText(
                    value = 7850,
                    min = 10,
                    max = 10000,
                    step = 10,
                    description = 'D (kg/m3)',
                    layout = ipw.Layout(width = width1, margin = margin1)
)
BoundedFloatText2 = ipw.BoundedFloatText(
                    value = 50,
                    min = 10,
                    max = 10000,
                    step = 0.005,
                    description = 'Ø ext. (mm)',
                    layout = ipw.Layout(width = width1, margin = margin1)
)
BoundedFloatText3 = ipw.BoundedFloatText(
                    value = 0,
                    min = 0,
                    max = 10000,
                    step = 0.005,
                    description = 'Ø int. (mm)',
                    layout = ipw.Layout(width = width1, margin = margin1)
)
BoundedFloatText4 = ipw.BoundedFloatText(
                    value = 400,
                    min = 10,
                    max = 10000,
                    step = 0.05,
                    description = 'L (mm)',
                    layout = ipw.Layout(width = width1, margin = margin1)
)
BoundedFloatText5 = ipw.BoundedIntText(
                    value = 0,
                    min = 0,
                    max = 1000,
                    step = 1,
                    description = 'Elementos',
                    layout = ipw.Layout(width = width1, margin = margin1)
)
FloatText2 = ipw.FloatText(
             value = -1,
             description = 'F (KN)',
             layout = ipw.Layout(width = width1, margin = margin1)
)
FloatText3 = ipw.FloatText(
             description = 'M (KN.m)',
             layout = ipw.Layout(width = width1, margin = margin1)
)
FloatSlider3 = ipw.FloatSlider(
               description = 'X',
               min = 0,
               max = BoundedFloatText4.value,
               value = BoundedFloatText4.value / 2,
               layout = ipw.Layout(margin = margin1)
)
Checkbox1 = ipw.Checkbox(
            value = False,
            description = 'Peso propio del eje',
            layout = ipw.Layout(width = width1, margin = margin1),
            indent = True
)
Button1 = ipw.Button(
          description = 'Calcular',
          button_style = 'success',
          icon = 'gear',
          layout = ipw.Layout(width = '20%', height = 'auto', margin = '0px 10px 0px 0px')
)
Button1.style.button_color = 'black'
Button2 = ipw.Button(
          description = 'Graficar',
          button_style = 'success',
          icon = 'pencil',
          layout = ipw.Layout(width = '20%', height = 'auto', margin = '0px 10px 0px 0px')
)
Button2.style.button_color = 'black'
Button3 = ipw.Button(
          description = 'Enumerar',
          button_style = 'success',
          icon = 'eye',
          layout = ipw.Layout(width = '20%', height = 'auto', margin = '0px 0px 0px 0px')
)
Button3.style.button_color = 'black'

grid1 = ipw.GridspecLayout(4, 3, layout = ipw.Layout(width = '100%', height = '120px', margin = '0px 0px 0px 0px', padding = '0px', border = '0px'))
for i in range(4):
    for j in range(3):
        grid1[i, j] = ipw.Text(layout = ipw.Layout(height = '5px', width = '265px'), disabled = True)
grid1[0, 1].value = 'Deflexiones (mm)'
grid1[0, 2].value = 'Velocidad Crítica (rpm)'
grid1[1, 0].value = 'Bernoulli'
grid1[2, 0].value = 'Timoshenko'
grid1[3, 0].value = '% de Error'

scale_x2 = bqp.LinearScale()
scale_y2 = bqp.LinearScale()
Lines1 = bqp.Lines(x = [], y = [],
         scales = {'x': scale_x2, 'y': scale_y2,},
         opacities = [1],
         colors = ['black'],
         stroke_width = 1.5,
         line_style = 'solid'
)
Lines2 = bqp.Lines(x = [], y = [],
         scales = {'x': scale_x2, 'y': scale_y2,},
         opacities = [1],
         colors = ['black'],
         stroke_width = 1.5,
         line_style = 'dashed'
)
scale_x3 = bqp.LinearScale()
scale_y3 = bqp.LinearScale()
Lines3 = bqp.Lines(x = [], y = [],
         scales = {'x': scale_x3, 'y': scale_y3,},
         opacities = [1],
         colors = ['black'],
         stroke_width = 2,
         line_style = 'solid',
         fill = 'inside',
         fill_opacities = [0.3],
         close_path = True,
         marker = 'circle',
         marker_size = 50
)
scale_x4 = bqp.LinearScale()
scale_y4 = bqp.LinearScale()
Lines4 = bqp.Lines(x = [], y = [],
         scales = {'x': scale_x4, 'y': scale_y4,},
         opacities = [1],
         colors = ['black'],
         stroke_width = 2,
         line_style = 'solid',
         fill = 'inside',
         fill_opacities = [0.3],
         close_path = True
)
Scatter1 = bqp.Scatter(x = [], y = [],
                   scales = {'x': scale_x2, 'y': scale_y2,},
                   opacities = [1],
                   colors = ['black'],
                   marker = 'triangle-up',
                   default_size = 250
)
Scatter2 = bqp.Scatter(x = [], y = [],
                   scales = {'x': scale_x2, 'y': scale_y2,},
                   opacities = [1],
                   colors = ['black'],
                   marker = 'square',
                   default_size = 250
)
Scatter3 = bqp.Scatter(x = [], y = [],
                   scales = {'x': scale_x2, 'y': scale_y2,},
                   opacities = [1],
                   colors = ['black'],
                   marker = 'triangle-down',
                   default_size = 100
)
Scatter4 = bqp.Scatter(x = [], y = [],
                   scales = {'x': scale_x4, 'y': scale_y4,},
                   opacities = [1],
                   colors = ['black'],
                   marker = 'circle',
                   default_size = 50,
                   unhovered_style = {'opacity': 0.5}
)

ax_x2   = bqp.Axis(scale = scale_x2, label = 'Longitud (mm)', label_color = 'black', label_offset = '-2em', grid_lines = 'none', tick_format = '0.0f', tick_style = {'stroke': 'black'}, color = 'black')
ax_y2   = bqp.Axis(scale = scale_y2, label = 'Diametro (mm)', label_color = 'black', label_offset = '-2em', grid_lines = 'none', tick_format = '0.0f', tick_style = {'stroke': 'black'}, color = 'black', orientation = 'vertical')
m_fig   = dict(left = 50, top = 20, bottom = 30, right = 30)
panzoom = bqp.interacts.PanZoom(scales={'x': [scale_x2], 'y': [scale_y2]})
fig3    = plt.figure(interaction = panzoom, title = 'Esquema del eje a analizar', title_style = {'font-size': '15px','fill': 'black'}, marks = [Lines1, Lines2, Scatter1, Scatter2, Scatter3], axes = [ax_x2, ax_y2], fig_margin = m_fig, layout=ipw.Layout(width = 'auto', height = '65%'))

ax_x3   = bqp.Axis(scale = scale_x3, label = 'Longitud (mm)', label_color = 'black', label_offset = '-1em', grid_lines = 'dashed', tick_format = '0.0f', tick_style = {'stroke': 'black'}, color = 'black')
ax_y3   = bqp.Axis(scale = scale_y3, label = 'Resultado', label_color = 'black', label_offset = '-1em', grid_lines = 'dashed', tick_format = '0.0f', tick_style = {'stroke': 'black'}, color = 'black', orientation = 'vertical')
m_fig   = dict(left = 50, top = 20, bottom = 30, right = 30)
fig4    = plt.figure(marks = [Lines3], axes = [ax_x3, ax_y3], fig_margin = m_fig, layout=ipw.Layout(width = 'auto', height = '100%'))

ax_x4   = bqp.Axis(scale = scale_x4, label = 'Longitud (mm)', label_color = 'black', label_offset = '-1em', grid_lines = 'dashed', tick_format = '0.0f', tick_style = {'stroke': 'black'}, color = 'black')
ax_y4   = bqp.Axis(scale = scale_y4, label = 'Resultado', label_color = 'black', label_offset = '-1em', grid_lines = 'dashed', tick_format = '0.0f', tick_style = {'stroke': 'black'}, color = 'black', orientation = 'vertical')
m_fig   = dict(left = 50, top = 20, bottom = 30, right = 30)
fig5    = plt.figure(marks = [Lines4, Scatter4], axes = [ax_x4, ax_y4], fig_margin = m_fig, layout=ipw.Layout(width = 'auto', height = '100%'))

HBox3 = ipw.HBox([Button1, Dropdown3, Button2, Button3], 
        layout = ipw.Layout(justify_content = 'space-around', width = 'auto', height = 'auto', margin = '0px', padding = '0px', border = '0px'))
VBox4 = ipw.VBox([HBox3, fig3, grid1], 
        layout = ipw.Layout(justify_content = 'space-around', width = 'auto', height = 'auto', margin = '0px', padding = '0px', border = '0px'))
VBox2 = ipw.VBox([HTML1, BoundedFloatText1, BoundedFloatText2, BoundedFloatText3, BoundedFloatText4, FloatText2, FloatText3, Dropdown1, Dropdown2, BoundedFloatText5, Checkbox1, FloatSlider3], 
        layout = ipw.Layout(justify_content = 'flex-start', width = 'auto', height = 'auto', margin = '0px', padding = '0px', border = '0px'))
HBox2 = ipw.HBox([VBox2, VBox4], layout = ipw.Layout(justify_content = 'space-around', width = '100%', height = '100%', margin = '0px', padding = '0px', border = '0px'))
display(HBox2)

In [5]:
%%html
<div style="text-align: center;">
    <h1 style="font-size: 50pt; color: black;"><b>Solución Analítica</b></h1>
</div>

In [6]:
HTML2      = ipw.HTML(value = '<div style="border-bottom: 1.5px solid #000;"></div>')
HTMLMath16 = ipw.HTMLMath(
    layout = ipw.Layout(margin = '0px', padding = '0px', border = '0px')
)
HTMLMath17 = ipw.HTMLMath(
    layout = ipw.Layout(margin = '0px', padding = '0px', border = '0px')
)
HTMLMath18 = ipw.HTMLMath(
    layout = ipw.Layout(margin = '0px', padding = '0px', border = '0px')
)
HTMLMath19 = ipw.HTMLMath(
    layout = ipw.Layout(margin = '0px', padding = '0px', border = '0px')
)
HTMLMath20 = ipw.HTMLMath(
    layout = ipw.Layout(margin = '0px', padding = '0px', border = '0px')
)
HTMLMath21 = ipw.HTMLMath(
    layout = ipw.Layout(margin = '0px', padding = '0px', border = '0px')
)
HTMLMath27 = ipw.HTMLMath(
    layout = ipw.Layout(margin = '0px', padding = '0px', border = '0px')
)
HTMLMath28 = ipw.HTMLMath(
    layout = ipw.Layout(margin = '0px', padding = '0px', border = '0px')
)
HTMLMath29 = ipw.HTMLMath(
    layout = ipw.Layout(margin = '0px', padding = '0px', border = '0px')
)
HTMLMath30 = ipw.HTMLMath(
    layout = ipw.Layout(margin = '0px', padding = '0px', border = '0px')
)
VBox5 = ipw.VBox([HTML2, ipw.HTMLMath(r"<font size = '3', font color = 'black', font face = 'Times New Roman'>Ecuacion de cortantes:"), HTMLMath27,
                  HTML2, ipw.HTMLMath(r"<font size = '3', font color = 'black', font face = 'Times New Roman'>Ecuacion de momentos:"), HTMLMath16, 
                  HTML2, ipw.HTMLMath(r"<font size = '3', font color = 'black', font face = 'Times New Roman'>Ecuacion de pendientes:"), HTMLMath28,
                  HTML2, ipw.HTMLMath(r"<font size = '3', font color = 'black', font face = 'Times New Roman'>Ecuacion de deflexiones:"), HTMLMath17, 
                  HTML2, ipw.HTMLMath(r"<font size = '3', font color = 'black', font face = 'Times New Roman'>Sistema de ecuaciones:"), HTMLMath18, 
                  HTML2, ipw.HTMLMath(r"<font size = '3', font color = 'black', font face = 'Times New Roman'>Resultados:"), HTMLMath19, 
                  HTML2, ipw.HTMLMath(r"<font size = '3', font color = 'black', font face = 'Times New Roman'>Ecuacion de deflexiones:"), HTMLMath20, 
                  HTML2, ipw.HTMLMath(r"<font size = '3', font color = 'black', font face = 'Times New Roman'>Ecuacion de pendientes:"), HTMLMath29,
                  HTML2, ipw.HTMLMath(r"<font size = '3', font color = 'black', font face = 'Times New Roman'>Ecuacion de momentos:"), HTMLMath21,
                  HTML2, ipw.HTMLMath(r"<font size = '3', font color = 'black', font face = 'Times New Roman'>Ecuacion de cortantes:"), HTMLMath30, HTML2], 
                  layout = ipw.Layout(justify_content = 'space-around', width = '100%', height = 'auto', margin = '0px', padding = '0px', border = '0px'))
display(VBox5)

In [7]:
%%html
<div style="text-align: center;">
    <h1 style="font-size: 50pt; color: black;"><b>Solución Numérica</b></h1>
</div>

In [8]:
Output1 = ipw.Output(
          layout = ipw.Layout(width = '60%', height = 'auto')
)
Output2 = ipw.Output(
          layout = ipw.Layout(width = 'auto', height = 'auto')
)
Output3 = ipw.Output(
          layout = ipw.Layout(width = 'auto', height = 'auto')
)
HTMLMath25 = ipw.HTMLMath(
             value = r"<font size = '10'>x",
             layout = ipw.Layout(width = 'auto', height = 'auto', margin = '0px', padding = '0px', border = '0px')
)
HTMLMath26 = ipw.HTMLMath(
             value = r"<font size = '10'>=",
             layout = ipw.Layout(width = 'auto', height = 'auto', margin = '0px', padding = '0px', border = '0px')
)
HBox4 = ipw.HBox([Output1, HTMLMath25, Output2, HTMLMath26, Output3], 
        layout = ipw.Layout(justify_content = 'space-between', align_items = 'center', width = '100%', height = '100%', margin = '0px', padding = '0px', border = '0px'))
display(HBox4)

In [9]:
%%html
<div style="text-align: center;">
    <h1 style="font-size: 50pt; color: black;"><b>Gráfica Analítica</b></h1>
</div>

In [10]:
VBox3 = ipw.VBox([fig4], 
        layout = ipw.Layout(justify_content = 'space-around', width = '100%', height = '300px', margin = '0px', padding = '0px', border = '0px'))
display(VBox3)

In [11]:
%%html
<div style="text-align: center;">
    <h1 style="font-size: 50pt; color: black;"><b>Gráfica Numérica</b></h1>
</div>

In [12]:
VBox4 = ipw.VBox([fig5], 
        layout = ipw.Layout(justify_content = 'space-around', width = '100%', height = '300px', margin = '0px', padding = '0px', border = '0px'))
display(VBox4)

In [13]:
def round_expr(expr, num_digits):
    return expr.xreplace({n : round(n, num_digits) for n in expr.atoms(Number)})

def count_elems(valor):
    if 'NDom' in globals():
        if Scatter4.names == None:
            Scatter4.names = np.arange(1, len(NDom) + 1)
Button3.on_click(count_elems)

def draw_shaft(DE, DI, L, AI, AD, FS3):
    Lines1.x = np.array([[-L / 2, L / 2, L / 2, -L / 2, -L / 2], [FS3 - L / 2, FS3 - L / 2]], dtype = 'object')
    Lines1.y = np.array([[-DE / 2, -DE / 2, DE / 2, DE / 2, -DE / 2], [DE / 2, DE]], dtype = 'object')
    Lines2.x = np.array([-L / 2, L / 2, L / 2, -L / 2, -L / 2])
    Lines2.y = np.array([-DI / 2, -DI / 2, DI / 2, DI / 2, -DI / 2]) 
    Scatter1.x = [-L / 2]
    Scatter1.y = [-DE / 2]
    Scatter2.x = [L / 2]
    Scatter2.y = [-DE / 2]
    Scatter3.x = [FS3 - L / 2]
    Scatter3.y = [DE / 2]
    if AI == 'Elastico':
        Scatter1.marker = 'triangle-up'
    else:
        Scatter1.marker = 'square'
    if AD == 'Elastico':
        Scatter2.marker = 'triangle-up'
    else:
        Scatter2.marker = 'square'

def upd_diamay(valor):
    draw_shaft(valor, BoundedFloatText3.value, BoundedFloatText4.value, Dropdown1.value, Dropdown2.value, FloatSlider3.value)
f_upd_diamay = ipw.interactive(upd_diamay, valor = BoundedFloatText2)

def upd_diamen(valor):
    draw_shaft(BoundedFloatText2.value, valor, BoundedFloatText4.value, Dropdown1.value, Dropdown2.value, FloatSlider3.value)
f_upd_diamen = ipw.interactive(upd_diamen, valor = BoundedFloatText3)

def upd_longit(valor):
    draw_shaft(BoundedFloatText2.value, BoundedFloatText3.value, valor, Dropdown1.value, Dropdown2.value, FloatSlider3.value)
    FloatSlider3.max = valor
f_upd_longit = ipw.interactive(upd_longit, valor = BoundedFloatText4)

def upd_lefapo(valor):
    draw_shaft(BoundedFloatText2.value, BoundedFloatText3.value, BoundedFloatText4.value, valor, Dropdown2.value, FloatSlider3.value)
f_upd_lefapo = ipw.interactive(upd_lefapo, valor = Dropdown1)

def upd_rigapo(valor):
    draw_shaft(BoundedFloatText2.value, BoundedFloatText3.value, BoundedFloatText4.value, Dropdown1.value, valor, FloatSlider3.value)
f_upd_rigapo = ipw.interactive(upd_rigapo, valor = Dropdown2)

def move_arrow(valor):
    draw_shaft(BoundedFloatText2.value, BoundedFloatText3.value, BoundedFloatText4.value, Dropdown1.value, Dropdown2.value, valor)
f_move_arrow = ipw.interactive(move_arrow, valor = FloatSlider3)

def num_calc(L, E, I, G):
    L = L
    E = E
    I = I
    G = G
    Nod_Tot = np.concatenate((np.array([0]), np.cumsum(L[0:2])))
    Nod_Ini = Nod_Tot[0:2]
    Nod_Fin = Nod_Tot[1:3]
    A   = (np.pi / 4) * ((BoundedFloatText2.value / 1000) ** 2 - (BoundedFloatText3.value / 1000) ** 2)       
    m   = np.repeat(BoundedFloatText3.value / BoundedFloatText2.value, 2)
    k   = (6 * (1 + m ** 2) ** 2) / (7 * (1 + m ** 2) ** 2 + 20 * m ** 2)
    p   = (12 * E * I) / (k * A * G * L ** 2)
    N_E = np.repeat(BoundedFloatText5.value + 1, 2)
    L_L = (Nod_Fin - Nod_Ini) / N_E
    I_I = np.repeat(I, 2)
    P_P = np.repeat(p, 2)
    E_E = np.repeat(E, 2)
    L1  = [np.zeros((1, N_E[0]))[0], np.zeros((1, N_E[1]))[0]]
    I1  = [np.zeros((1, N_E[0]))[0], np.zeros((1, N_E[1]))[0]]
    E1  = [np.zeros((1, N_E[0]))[0], np.zeros((1, N_E[1]))[0]]
    phi1 = [np.zeros((1, N_E[0]))[0], np.zeros((1, N_E[1]))[0]]
    for i in range(2):
        L1[i] = np.repeat(L_L[i], N_E[i])
        I1[i] = np.repeat(I_I[i], N_E[i])
        E1[i] = np.repeat(E_E[i], N_E[i])
        phi1[i] = np.repeat(P_P[i], N_E[i])
    L1   = np.concatenate((L1[0], L1[1]))
    I1   = np.concatenate((I1[0], I1[1]))
    E1   = np.concatenate((E1[0], E1[1]))
    phi1 = np.concatenate((phi1[0], phi1[1]))
    MG   = np.zeros((sum(N_E), 2 * (sum(N_E) + 1), 2 * (sum(N_E) + 1)))
    ML   = np.zeros((sum(N_E), 4, 4))
    for i in range(sum(N_E)):
        ML[i, :, :] = (E1[i] / L1[i] ** 3) * (I1[i] / (1 + phi1[i])) * np.array([
                      [12, 6 * L1[i], -12, 6 * L1[i]],
                      [6 * L1[i], (4 + phi1[i]) * L1[i] ** 2, -6 * L1[i], (2 - phi1[i]) * L1[i] ** 2],
                      [-12, -6 * L1[i], 12, -6 * L1[i]],
                      [6 * L1[i], (2 - phi1[i]) * L1[i] ** 2, -6 * L1[i], (4 + phi1[i]) * L1[i] ** 2]])
        MG[i, np.array([2 * i, 2 * i + 1, 2 * i + 2, 2 * i + 3])[:,np.newaxis], np.array([2 * i, 2 * i + 1, 2 * i + 2, 2 * i + 3])[np.newaxis,:]] = ML[i, :, :]
    MG = np.sum(MG, axis = 0)

    FCT = np.arange(0, (3 + 2 * BoundedFloatText5.value) * 2)
    FCE = np.array([0, (2 + 2 * BoundedFloatText5.value) * 2])
    if Dropdown1.value == 'Fijo':
        FCE = np.concatenate((FCE, np.array([1])))
    if Dropdown2.value == 'Fijo':
        FCE = np.concatenate((FCE, np.array([(2 + 2 * BoundedFloatText5.value) * 2 + 1])))
    FCS = np.setdiff1d(FCT, FCE)
    MRA = MG[FCS[:,np.newaxis], FCS[np.newaxis,:]]
    if Checkbox1.value == True:
        FDN = - (((np.pi / 4) * ((BoundedFloatText2.value / 1000) ** 2 - (BoundedFloatText3.value / 1000) ** 2)) * (BoundedFloatText4.value / 1000) * BoundedFloatText1.value * 9.81) / (3 + 2 * BoundedFloatText5.value)
    else:
        FDN = 0
    MFD = np.repeat(FDN, 3 + 2 * BoundedFloatText5.value).reshape((3 + 2 * BoundedFloatText5.value, 1))
    MDF = np.zeros((3 + 2 * BoundedFloatText5.value, 1))
    MDF[1 + BoundedFloatText5.value] = FloatText2.value * 1000
    MDF = MDF + MFD
    MDM = np.zeros((3 + 2 * BoundedFloatText5.value, 1))
    MDM[1 + BoundedFloatText5.value] = FloatText3.value * 1000
    MFM = np.hstack((MDF, MDM)).reshape((len(MFD) * 2, 1))
    MRB = MFM[FCS]
    MRX = np.linalg.solve(MRA, MRB)
    MPD = np.zeros(((3 + 2 * BoundedFloatText5.value) * 2, 1))
    MPD[FCS] = MRX
    MFM = np.dot(MG, MPD)
    FC = np.array([])
    MF = np.array([])
    for i in range(sum(N_E)):
        MLR = np.dot(ML[i], MPD[(2 * i):(2 * i + 4)])
        FC = np.concatenate((FC, MLR[0], MLR[2]))
        MF = np.concatenate((MF, MLR[1], MLR[3]))
    RD = MPD[range(0, len(MPD), 2)]
    RP = MPD[range(1, len(MPD), 2)]
    Dom = np.unique(np.concatenate((np.linspace(0, FloatSlider3.value, BoundedFloatText5.value + 2), np.linspace(FloatSlider3.value, BoundedFloatText4.value, BoundedFloatText5.value + 2))))
    return RD, RP, MF, FC, MRA, MRB, MRX, Dom
        
def sym_calc(L, EI, AE, AF, F, M, D):
    Lon_Tra = L
    Ine_You = EI
    Apo_Ela = AE
    Apo_Fij = AF
    Fue_Nod = F
    Mom_Nod = M
    Fue_Dis = D

    Nod_Tot = np.concatenate((np.array([0]), np.cumsum(Lon_Tra[0:2])))
    Nod_Ini = Nod_Tot[0:2]
    Nod_Fin = Nod_Tot[1:3]
    Apo_Tot = Apo_Ela + Apo_Fij

    nvr = ''
    for i in range(1,np.shape(np.nonzero(Apo_Tot))[1] + 1):
            var = ' r' + str(i)
            nvr = nvr + var
    Cons_0  = sp.var(nvr)
    Rea_Fue = np.array([0, 0 ,0], dtype = object)
    Rea_Fue[Apo_Tot > 0] = Cons_0
    Fue_Nod = Fue_Nod + Rea_Fue

    if sum(Apo_Fij) != 0:
        nvr = ''
        for i in range(1,np.shape(np.nonzero(Apo_Fij))[1] + 1):
                var = ' m' + str(i)
                nvr = nvr + var
        Cons_1  = sp.var(nvr)
        Rea_Mom = np.array([0, 0 ,0], dtype = object)
        Rea_Mom[Apo_Fij > 0] = Cons_1
        Mom_Nod = Mom_Nod + Rea_Mom

    x = sp.var('x'); x1 = sp.var('x1'); x2 = sp.var('x2')
    Com_Fue = np.array([0, 0], dtype = object)
    Com_Dis = np.array([0, 0], dtype = object)
    for i in range(2):
        Com_Fue[i] = sum(Fue_Nod[0:i + 1])
        Com_Dis[i] = Fue_Dis[i] * (x - Nod_Ini[i])
    Fue_Equ = Fue_Dis * (Nod_Fin - Nod_Ini)
    Fue_Equ = np.concatenate((np.array([0]), np.cumsum(Fue_Equ[0:1])))

    Ecu_Cor = np.array([0, 0], dtype = object)
    for i in range(2):
        Ecu_Cor[i] = Com_Fue[i] + Com_Dis[i] + Fue_Equ[i]
    
    x1 = 0
    Ecu_Mom = np.array([0, 0], dtype = object)
    for i in range(2):
        Ecu_1      = sp.integrate(Ecu_Cor[i], x) + x2 + Mom_Nod[i] - x1
        Ecu_1      = Ecu_1.subs(x, Nod_Ini[i])
        Ecu_2      = sp.integrate(Ecu_Cor[i], x) + np.array(solve(Ecu_1, x2))[0]
        Ecu_Mom[i] = Ecu_2
        x1         = Ecu_2.subs(x, Nod_Fin[i])      
    Sis_Fue = sum(Fue_Nod) + sum(Fue_Dis * (Nod_Fin - Nod_Ini))
    Sis_Mom = sum(Mom_Nod) + sum(Nod_Fin * Fue_Nod[1:]) + sum((Fue_Dis * (Nod_Fin - Nod_Ini)) * ((Nod_Fin + Nod_Ini) / 2))

    Ecu_Pen = np.array([0, 0], dtype = object)
    Ecu_Def = np.array([0, 0], dtype = object)
    for i in range(2):
        Ecu_Pen[i] = sp.integrate(Ecu_Mom[i], x) + sp.var('c' + str(2 * (i + 1) - 1))
        Ecu_Def[i] = sp.integrate(Ecu_Pen[i], x) + sp.var('c' + str(2 * (i + 1)))
    
    if FloatSlider3.value <= BoundedFloatText4.value / 2:
        Sho_Cor = Ecu_Cor[0]
        Sho_Mom = Ecu_Mom[0] 
        Sho_Pen = Ecu_Pen[0]
        Sho_Def = Ecu_Def[0]
    else:
        Sho_Cor = Ecu_Cor[1]
        Sho_Mom = Ecu_Mom[1] 
        Sho_Pen = Ecu_Pen[1]
        Sho_Def = Ecu_Def[1]
    
    Sis_Pen = Ecu_Pen[1].subs(x, Nod_Fin[0]) / Ine_You - Ecu_Pen[0].subs(x, Nod_Fin[0]) / Ine_You
    Sis_Def = Ecu_Def[1].subs(x, Nod_Fin[0]) / Ine_You - Ecu_Def[0].subs(x, Nod_Fin[0]) / Ine_You

    Sis_Nod = np.array([0, 0], dtype = object)
    Sis_Nod[0] = Ecu_Def[0].subs(x, Nod_Tot[0])
    Sis_Nod[1] = Ecu_Def[1].subs(x, Nod_Tot[2])
    if sum(Apo_Fij) != 0: 
        Sis_Nop = np.zeros((1, np.count_nonzero(Apo_Fij)), dtype = object)[0]
        j = -1
        for i in range(3):
            if Apo_Fij[i] != 0:
                j = j + 1
                if Apo_Fij[i] == 1:
                    Sis_Nop[j] = Ecu_Pen[0].subs(x, Nod_Tot[i])
                elif Apo_Fij[i] == 3:
                    Sis_Nop[j] = Ecu_Pen[1].subs(x, Nod_Tot[i])
        if len(Sis_Nop) == 1:
            Sis_Ecu = (Sis_Fue, Sis_Mom, Sis_Pen, Sis_Def, Sis_Nod[0], Sis_Nod[1], Sis_Nop[0])
        else:
            Sis_Ecu = (Sis_Fue, Sis_Mom, Sis_Pen, Sis_Def, Sis_Nod[0], Sis_Nod[1], Sis_Nop[0], Sis_Nop[1])
    else:
        Sis_Ecu = (Sis_Fue, Sis_Mom, Sis_Pen, Sis_Def, Sis_Nod[0], Sis_Nod[1])
    rpts = sp.solve(Sis_Ecu)

    for i in range(2):
        Ecu_Def[i] = Ecu_Def[i].subs(c1, rpts[c1])
        Ecu_Def[i] = Ecu_Def[i].subs(c2, rpts[c2])
        Ecu_Def[i] = Ecu_Def[i].subs(c3, rpts[c3])
        Ecu_Def[i] = Ecu_Def[i].subs(c4, rpts[c4])
        Ecu_Def[i] = Ecu_Def[i].subs(r1, rpts[r1])
        if sum(Apo_Fij) != 0: 
            Ecu_Def[i] = Ecu_Def[i].subs(m1, rpts[m1])

    Ecu_Def = Ecu_Def / Ine_You
    for i in range(2):
        Ecu_Pen[i] = sp.diff(Ecu_Def[i], x)
        Ecu_Mom[i] = sp.diff(Ecu_Pen[i], x) * Ine_You
        Ecu_Cor[i] = sp.diff(Ecu_Mom[i], x)

    Dom = np.array([np.linspace(Nod_Tot[0], Nod_Tot[1], 50), np.linspace(Nod_Tot[1], Nod_Tot[2], 50)])
    Val_Def = np.array([])
    Val_Pen = np.array([])
    Val_Mom = np.array([])
    Val_Cor = np.array([])
    for i in range(2):
        flamb = lambdify(x, Ecu_Def[i], "numpy")
        Val_Def = np.append(Val_Def, flamb(Dom[i]))
        flamb = lambdify(x, Ecu_Pen[i], "numpy")
        Val_Pen = np.append(Val_Pen, flamb(Dom[i]))
        flamb = lambdify(x, Ecu_Mom[i], "numpy")
        Val_Mom = np.append(Val_Mom, flamb(Dom[i]))
        flamb = lambdify(x, Ecu_Cor[i], "numpy")
        Val_Cor = np.append(Val_Cor, flamb(Dom[i]))
    Dom = Dom.reshape(100,) * 1000
    
    if FloatSlider3.value <= BoundedFloatText4.value / 2:
        Hid_Cor = Ecu_Cor[0]
        Hid_Mom = Ecu_Mom[0] 
        Hid_Pen = Ecu_Pen[0]
        Hid_Def = Ecu_Def[0]
    else:
        Hid_Cor = Ecu_Cor[1]
        Hid_Mom = Ecu_Mom[1] 
        Hid_Pen = Ecu_Pen[1]
        Hid_Def = Ecu_Def[1]
    
    return Val_Def, Val_Pen, Val_Mom, Val_Cor, Sho_Cor, Sho_Mom, Sho_Pen, Sho_Def, Sis_Ecu, rpts, Hid_Def, Hid_Pen, Hid_Mom, Hid_Cor, Dom

def shaft_calc(valor):
    global Sim_Def, Sim_Pen, Sim_Mom, Sim_Cor, Dom, Num_Def, Num_Pen, Num_Mom, Num_Cor, NDom, Fun_Def
    L = np.array([FloatSlider3.value / 1000, (BoundedFloatText4.value - FloatSlider3.value) / 1000])
    EI = 207e9 * ((np.pi / 64) * ((BoundedFloatText2.value / 1000) ** 4 - (BoundedFloatText3.value / 1000) ** 4))
    AE = np.array([0, 0, 0])
    AF = np.array([0, 0, 0])
    if Dropdown1.value == 'Elastico':
        AE[0] = 1
    else:
        AF[0] = 1
    if Dropdown2.value == 'Elastico':
        AE[2] = 3
    else:
        AF[2] = 3
    F = np.array([0, FloatText2.value * 1000, 0])
    M = np.array([0, FloatText3.value * 1000, 0])
    if Checkbox1.value == True:
        D = ((np.pi / 4) * ((BoundedFloatText2.value / 1000) ** 2 - (BoundedFloatText3.value / 1000) ** 2)) * BoundedFloatText1.value * 9.81
        D = np.array([-D, -D])
    else:
        D = np.array([0, 0])
    Sim_Def, Sim_Pen, Sim_Mom, Sim_Cor, Sho_Cor, Sho_Mom, Sho_Pen, Sho_Def, Sis_Ecu, rpts, Ecu_Def, Ecu_Pen, Ecu_Mom, Ecu_Cor, Dom = sym_calc(L, EI, AE, AF, F, M, D)
    HTMLMath27.value = f'${latex(Sho_Cor)}$'
    HTMLMath16.value = f'${latex(Sho_Mom)}$'
    HTMLMath28.value = f'${latex(Sho_Pen)}$'
    HTMLMath17.value = f'${latex(Sho_Def)}$'
    HTMLMath18.value = f'${latex(Sis_Ecu)}$'
    HTMLMath19.value = f'${latex(rpts)}$'
    HTMLMath20.value = f'${latex(Ecu_Def)}$'
    HTMLMath29.value = f'${latex(Ecu_Pen)}$'
    HTMLMath21.value = f'${latex(Ecu_Mom)}$'
    HTMLMath30.value = f'${latex(Ecu_Cor)}$'
    
    E  = 207e9
    I  = (np.pi / 64) * ((BoundedFloatText2.value / 1000) ** 4 - (BoundedFloatText3.value / 1000) ** 4)
    G   = 80e9
    Num_Def, Num_Pen, Num_Mom, Num_Cor, MRA, MRB, MRX, NDom = num_calc(L, E, I, G) 
    df1 = pd.DataFrame(MRA)
    df1_html = df1.to_html(index = False, header = False)
    df2 = pd.DataFrame(MRB)
    df2_html = df2.to_html(index = False, header = False)
    df3 = pd.DataFrame(MRX)
    df3_html = df3.to_html(index = False, header = False)
    with Output1:
        display(HTML(df1_html))
    with Output2:
        display(HTML(df2_html))
    with Output3:
        display(HTML(df3_html))
    Output1.clear_output(wait = True)
    Output2.clear_output(wait = True)
    Output3.clear_output(wait = True)
        
    X = Dom
    Y = abs(Sim_Def)
    S1 = np.trapz(y = Y, x = X)
    S2 = np.trapz(y = Y ** 2, x = X)
    Sim_Vcr = (30 / np.pi) * np.sqrt(9.81 * abs(S1/ S2))
    # S1 = abs(sp.integrate(Fun_Def[0], (x, 0, FloatSlider3.value / 1000))) + abs(sp.integrate(Fun_Def[1], (x, FloatSlider3.value / 1000, BoundedFloatText4.value / 1000)))
    # S2 = abs(sp.integrate(Fun_Def[0] ** 2, (x, 0, FloatSlider3.value / 1000))) + abs(sp.integrate(Fun_Def[1] ** 2, (x, FloatSlider3.value / 1000, BoundedFloatText4.value / 1000)))
    # Sim_Vcr = (30 / np.pi) * sp.sqrt(9.81 * abs(S1/ S2))
    X = NDom
    Y = abs(Num_Def.T[0])
    S1 = np.trapz(y = Y, x = X)
    S2 = np.trapz(y = Y ** 2, x = X)
    Num_Vcr = (30 / np.pi) * np.sqrt(9.81 * abs(S1/ S2))
    if Checkbox1.value == False:
        grid1[1, 1].value = str(max(abs(Sim_Def)))
        grid1[1, 2].value = str(Sim_Vcr)
        grid1[2, 1].value = str(max(abs(Num_Def.T[0])))
        grid1[2, 2].value = str(Num_Vcr)
        grid1[3, 1].value = str(abs(((max(abs(Sim_Def)) - max(abs(Num_Def.T[0]))) / max(abs(Num_Def.T[0]))) * 100))
        grid1[3, 2].value = str(abs(((Sim_Vcr - Num_Vcr) / Num_Vcr) * 100))
Button1.on_click(shaft_calc)

def draw_graph(valor):
    if Dropdown3.value == 'Cortantes':
        ylabel = 'Cortantes (N)'
        if Checkbox1.value == False:
            YDom = np.repeat(Sim_Cor, 50)
        else:
            YDom = Sim_Cor
        DomY = Num_Cor[range(0, len(Num_Cor), 2)]
        DomY = np.append(DomY, -Num_Cor[-1])
    elif Dropdown3.value == 'Momentos':
        ylabel = 'Momentos (N.m)'
        YDom = Sim_Mom
        DomY = Num_Mom[range(0, len(Num_Mom), 2)]
        DomY = -np.append(DomY, -Num_Mom[-1])
    elif Dropdown3.value == 'Pendientes':
        ylabel = 'Pendientes (rad)'
        YDom = Sim_Pen
        DomY = Num_Pen.T[0]
    elif Dropdown3.value == 'Deflexiones':
        ylabel = 'Deflexiones (mm)'
        YDom = Sim_Def * 1000
        DomY = Num_Def.T[0] * 1000
    Scatter4.x = NDom
    Scatter4.y = DomY
    Scatter4.tooltip = bqp.Tooltip(fields = ['x', 'y'], formats = ['.2f', '.2f'])
    Scatter4.names = None
    XDom = np.insert(Dom, 0, 0)
    XDom = np.append(XDom, BoundedFloatText4.value)
    YDom = np.insert(YDom, 0, 0)
    YDom = np.append(YDom, 0)
    DomX = np.insert(NDom, 0, 0)
    DomX = np.append(DomX, BoundedFloatText4.value)
    DomY = np.insert(DomY, 0, 0)
    DomY = np.append(DomY, 0)
    Lines3.x = XDom
    Lines3.y = YDom
    Lines3.tooltip = bqp.Tooltip(fields = ['x', 'y'], formats = ['.2f', '.2f'])
    Lines4.x = DomX
    Lines4.y = DomY
    fig4.axes[1].label = ylabel
    fig5.axes[1].label = ylabel
Button2.on_click(draw_graph)